# Bachelor's thesis: Naive Bayes EM model

### The corresponding paper is: Anderson, Billie. "Naive Bayes using the expectation-maximization algorithm for reject inference." Communications in Statistics: Case Studies, Data Analysis and Applications 8.3 (2022): 484-504.

In [1]:
# Laden von Standard-Bibliotheken
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score,roc_curve, brier_score_loss
from sklearn.naive_bayes import GaussianNB

## Loading the data 

In [2]:
accepts = pd.DataFrame()
accepts = pd.read_csv('../data/New_accepts.csv',encoding = "ISO-8859-1", low_memory=False)
rejects = pd.DataFrame()
rejects = pd.read_csv('../data/New_rejects.csv',encoding = "ISO-8859-1", low_memory=False)

In [3]:
X_acc = accepts.copy()
X_rej = rejects.copy()
y_rej = X_rej.pop("loan_status")
y_acc = X_acc.pop("loan_status")

## Preprocessing

### We preprocess the data as per the paper with 4 bins per feature and accepts and rejects binned seperately

In [4]:
def categorize_emp_length(x):
    if(x== 0):
        return 0
    if(x== 1 or x== 2 or x== 3 or x== 4 or x == 5):
        return 1
    if(x==6 or x== 7 or x== 8 or x== 9):
        return 2
    if(x== 10):
        return 3

In [5]:
X_acc['emp_length'] = X_acc['emp_length'].apply(categorize_emp_length)
X_rej['emp_length'] = X_rej['emp_length'].apply(categorize_emp_length)

In [6]:
X_acc['loan_amnt'] = pd.qcut(X_acc['loan_amnt'], 4, labels=False)
X_acc['dti'] = pd.qcut(X_acc['dti'], 4, labels=False)
X_acc['fico'] = pd.qcut(X_acc['fico'], 4, labels=False) 

In [7]:
X_rej['loan_amnt'] = pd.qcut(X_rej['loan_amnt'], 4, labels=False)
X_rej['dti'] = pd.qcut(X_rej['dti'], 4, labels=False)
X_rej['fico'] = pd.qcut(X_rej['fico'], 4, labels=False)

In [8]:
X_acc['loan_amnt'] = X_acc['loan_amnt'].astype('category')
X_acc['dti'] = X_acc['dti'].astype('category')
X_acc['fico']= X_acc['fico'].astype('category')
X_acc['emp_length']= X_acc['emp_length'].astype('category')

In [9]:
X_rej['loan_amnt'] = X_rej['loan_amnt'].astype('category')
X_rej['dti'] = X_rej['dti'].astype('category')
X_rej['fico']= X_rej['fico'].astype('category')
X_rej['emp_length']= X_rej['emp_length'].astype('category')

In [10]:
X_acc = pd.get_dummies(data=X_acc, columns = ['loan_amnt'], drop_first = False)
X_acc = pd.get_dummies(data=X_acc, columns = ['dti'], drop_first = False)
X_acc = pd.get_dummies(data=X_acc, columns = ['fico'], drop_first = False)
X_acc = pd.get_dummies(data=X_acc, columns = ['emp_length'], drop_first = False)
X_acc = pd.get_dummies(data=X_acc, columns = ['addr_state'], drop_first = False)

In [11]:
X_rej = pd.get_dummies(data=X_rej, columns = ['loan_amnt'], drop_first = False)
X_rej = pd.get_dummies(data=X_rej, columns = ['dti'], drop_first = False)
X_rej = pd.get_dummies(data=X_rej, columns = ['fico'], drop_first = False)
X_rej = pd.get_dummies(data=X_rej, columns = ['emp_length'], drop_first = False)
X_rej = pd.get_dummies(data=X_rej, columns = ['addr_state'], drop_first = False)

### We also need to set up a list for the parameters. Those are the estimated thetas, variances and prior class probabilities which we will average over the bootstrapping.

In [12]:
parameter_estimates_theta = []
parameter_estimates_var = []
parameter_estimates_class_prior = []

### Here you can chose the parameters, which are the fraction of accepts and rejects used in the Bootstrapping. The paper suggested 1/4 of the accepts and 1/6 of the rejects. We also need to set the value for the amount of Bootstraps we do.

In [13]:
fraction_accepts = 1/6
fraction_rejects = 1/4
Bootstrap = 200

## Bootstrapping to obtain parameters

In [14]:
for b in range(Bootstrap):
            
            
    #1. Data Preprocessing
            
    accept = pd.concat([X_acc, y_acc], axis = 1)
    accept_sampled = accept.sample(frac = fraction_accepts, replace = True)
    reject_sampled = X_rej.sample(frac = fraction_rejects, replace = True)
    y_acc_sampled = accept_sampled.pop("loan_status") 
            
    #2. Build Naive Bayes model using accepted data
            
    first_model = GaussianNB()
            
    first_model.fit(accept_sampled, y_acc_sampled)

    #3. Predict labels for rejected applicants

    y_rejected_pred = first_model.predict_proba(reject_sampled)
    y_rejected_pred = y_rejected_pred[:,1]
    y_rejected_pred = [int(i >= 0.5) for i in y_rejected_pred]

    #4. Build a new model using accepts and rejects with inferred labels

    second_model=GaussianNB()
    X_combined = np.concatenate((accept_sampled, reject_sampled))
    y_combined = np.concatenate((y_acc_sampled, y_rejected_pred))
            
    #print(np.shape(X_combined))
            
    second_model.fit(X_combined, y_combined)

    #print(nb_model.theta_)
    #print(np.shape(y_combined))
    #print(second_model.theta_)
    #print(second_model.var_)
            
    # Store parameter estimates      
    parameter_estimates_theta.append(second_model.theta_)
    parameter_estimates_var.append(second_model.var_)
    parameter_estimates_class_prior.append(second_model.class_prior_)
            


### We have 5 features and 4 bins per feature meaning 5 * 4 = 20 features in total. Our Bayesian model optimizes 2 parameters per feature meaning we need a list of 20 * 2. We need 2 lists for theta and variance, as well as a parameter for the prior probabilities.

In [15]:
shape = (2, 20)
parameter_theta = np.zeros(shape)
parameter_var = np.zeros(shape)
parameter_class_prior = [0, 0]

### Now we average the estimates from the Bootstrapping procedure

In [16]:
for l in range (2):
    for j in range (20):
        for i in range (Bootstrap):
            parameter_theta[l][j] = parameter_theta[l][j] + parameter_estimates_theta[i][l][j]

In [17]:
for l in range (2):
    for j in range (20):
        for i in range (Bootstrap):
            parameter_var[l][j] = parameter_var[l][j] + parameter_estimates_var[i][l][j]

In [18]:
for l in range (2):
    for j in range (20):
        parameter_theta[l][j] = parameter_theta[l][j] / (Bootstrap)

In [19]:
for l in range (2):
    for j in range (20):
        parameter_var[l][j] = parameter_var[l][j] / (Bootstrap)

In [20]:
for j in range (2):
    for i in range (Bootstrap):
        parameter_class_prior[j] = parameter_class_prior[j] +  parameter_estimates_class_prior[i][j]

In [21]:
for j in range (2):
    parameter_class_prior[j] = parameter_class_prior[j] / (Bootstrap)

## Final Model

In [22]:
gnb = GaussianNB()
gnb.theta_ = parameter_theta
gnb.var_ = parameter_var
gnb.class_count_ = np.array([459005, 93530])  
gnb.class_prior_ = parameter_class_prior
gnb.classes_ = np.array([0, 1])

In [23]:
yhat = gnb.predict_proba(X_rej)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


## Results

In [24]:
auc = roc_auc_score(y_rej,yhat[:,1])
print("AUC:", auc)

AUC: 0.5133459117323874


In [25]:
brier = brier_score_loss(y_rej,yhat[:,1],pos_label=1)
print("Brier score:", brier)

Brier score: 0.5609473127067169


In [26]:
fpr, tpr, thresholds = roc_curve(y_rej,yhat[:,1]) 
ks_statistic = max(tpr - fpr)
print("KS-Statistic:",ks_statistic)

KS-Statistic: 0.02350272441458101
